- [Paper: Optimal Online Adaptive Electric Vehicle Charging](http://netlab.caltech.edu/assets/publications/Guo-2017-OLP.pdf)
- [Pyomo - Optimization Modeling in Python](https://pyomo.readthedocs.io/en/stable/)
- [ TeX functions supported by KaTeX](https://katex.org/docs/supported.html)
- [TGC_LP: OLP in Excel for Enexis (OneDrive)](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

# OLP: Online Linear Programming

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!! 
    <BR>
    >  Check: <strong>objective outcome</strong> and the <strong>Power</strong> (Variables) 
    <BR>
    <BR>
</div>

### ✔ Pyomo-0

#### Model

This first model has only 1 EV/SE and has to determine the ``kW`` for the next ``n`` periods (n=5).  

The objective is to minimize the difference in the  ``theoretical maximum energy output [kWh]``, being ```Enexis max power [kW] * duration [h]``` and the ``actual delivered energy``, being ```actual power [kW] * duration [h]```.

By normalizing the duration, the resulting ``weighted-% energy delivered`` can easily be combined with any future objective like the ``weighted-% customer satisfaction``. 

This results in the following ``Objective function`` to be minimalized. 


##### Objective Function


$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{x_{j}}{ex_{j}} \right), \\
\text{where} \; &w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \; \forall j, k \in \{1, 2, \ldots, n\} \\
\end{align*}
$$

$ \text{Note: } j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}$ 

##### Decision Variables

The charging power at time slot $j$ are the decision variables, $x_{j}$ 

##### Constraints

The charging power to be used ($x_{j}$) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llllll}
\text{s.t.} & x_{j} \geq 0  &  \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ex_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq ev_{j} & \forall j \in \{1, 2, \ldots, n\} \\
& x_{j} \leq se_{j} & \forall j \in \{1, 2, \ldots, n\} \\
\end{array} 
$$

$ \text{Note: } j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}$ 

This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_0``) 

<img src="./images/pyomo_0.png" width="1000">

#### Code & Results

##### Session: EV/EVSE

In [ ]:
import numpy as np

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period


EV01 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE01 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES01 = np.minimum(EV01, SE01)

EVSE = np.array([ES01])

##### TGC: Central Controller

In [ ]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

# Create the weights w[j] for the parking
pt = np.array([40, 10, 20, 10, 10])
w = pt / np.sum(pt)

# row reserved for future use
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()


# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (j + 1): EVSE[i][j]
    for i in range(len(EVSE))
    for j in range(len(EVSE[i]))
}
model.session = Param(model.J, initialize=session_max_kw)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------

def obj_expression(model):
    # return the expression for the objective
    return sum(w[j - 1] * (1 - model.x[j] / model.enexis[j]) for j in model.J)



model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return model.x[j] <= model.enexis[j]


# is ev/se power <= ev max power input for period j?
def session_constraint_rule(model, j):
    return model.x[j] <= model.session[j]


# the next lines create one constraint for each member of the set model.J
model.Enexis__Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.Session_Constraint = Constraint(model.J, rule=session_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power EVSE 1: {[round(tgc.x[x].value, 2) for x in range(1, 6)]}")


print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

### ✔ Pyomo-1

#### Model

Next, we extend the ``pyomo-0`` model by allowing multiple EV/SE parallel charging sessions, (m = 3).  
This results in the following ``Objective function`` to be minimized.

##### Objective Function


<!-- $$
\begin{array}{llll}
\text{minimize} \; \sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right), \\\\
\text{where} \; w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad \\\\
\forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\} 
\end{array} 
$$ -->

$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right), \\\\
\text{where} \; 
&w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \\\\
&\forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}
\end{array} 
$$


##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($ x_{ij} $) in kW is subject to 4 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.


$$
\begin{array}{llll}
\text{s.t.} & x_{ij} \geq 0  &  \\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} & \\
& x_{ij} \leq ev_{ij} & \\
& x_{ij} \leq se_{ij} & \\ 
\end{array} 
$$



$$
\begin{array}{ll}
\forall i \in \{1, 2, \ldots, m,\} & \text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\} &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_1``) 


<img src="./images/pyomo_1.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [ ]:
import numpy as np

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES01 = np.minimum(EV01, SE01)

EV02 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE02 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES02 = np.minimum(EV02, SE02)

EV03 = np.array([7.36, 7.36, 5.00, 3.68, 0.00])
SE03 = np.array([5.00, 7.36, 5.00, 3.68, 0.00])
ES03 = np.minimum(EV03, SE03)

EVSE = np.array([ES01, ES02, ES03])


##### TGC: Central Controller

In [ ]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j]
    for i in range(len(EVSE))
    for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(w[j-1] * (1 - sum(model.x[i, j] for i in model.I)/model.enexis[j]) 
               for j in model.J)

model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= ev max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# the next line creates one constraint for each member of the set model.J
model.Enexis__Constraintt = Constraint(model.J, rule=demand_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)


# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power EVSE 1: {[round(tgc.x[1, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 2: {[round(tgc.x[2, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 3: {[round(tgc.x[3, x].value, 2) for x in range(1, 6)]}")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

### ✔ Pyomo-2

<br>

<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>‼ Note:</strong>
    </div>
    <BR>
    >  The <strong>objective outcome</strong> of the excel-model is the same as the pyomo-model !!! 
    <BR>
    >  The <strong>decision variable</strong> are different though!! (multiple solutions seem to be possible) 
    <BR>
    <BR>
</div>

#### Model

The Enexis Grid Connection (ECG) utilization (%) is the highest priority, as modelled in ``pyomo-1``.
 
Next, the customer satisfaction is taken into account as second priority. This will be measured as % of ``desired charge`` $dc_{i}$ which has been delivered at the time the customer leaves. The ``final charge`` $fc_{i}$ in kWh can be calculated when we know for each EV the parking time ($pt_{i}$).

$$
fc_{i} = \displaystyle\sum_{j=1}^{n}  pt_{j}  \; x_{ij}
$$

Please note that the model shouldn't be penalized for unrealistic customer demands. The customers ``desired charge`` $dc_{i}$ in (kWh) may never be higher than the ``realistc charge`` $rc_{i}$, being the maximum energy which can be deliverd at maximum charging power during the time of parking!  

$$
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}) \\
$$

The customer satisfaction will be expressed as: 

$$
\text{customer satisfaction}_{i} =  \frac{fc_{i}}{rc_{i}} \; \\\\
$$

$$ \forall i \in \{1, 2, \ldots, m\}, \; \forall j \in \{1, 2, \ldots, n\} $$

##### Objective Function

Modifying the ``pyomo-1`` model results in the following ``Objective function`` to be minimized:  
<br>
<br>

$$
\begin{align*}
\text{minimize} \; &\sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right) + \sum_{i=1}^{m} \left(1 - \frac{fc_{i}}{rc_{i}}  \right), \\\\
\text{where} \; 
& w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad 
fc_{i} = \sum_{j=1}^{n} pt_{j} x_{ij}, \quad 
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}), \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.}
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} &\\
& x_{ij} \leq ev_{ij} &\\
& x_{ij} \leq se_{ij} &\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & 
\end{array} 
$$



$$
\begin{array}{ll}
\forall i \in \{1, 2, \ldots, m,\} & \text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\} &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_2``) 

<img src="./images/pyomo_2.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [ ]:
import numpy as np

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([3.68, 7.36, 7.36, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES01 = np.minimum(EV01, SE01)
# print(ES01)

EV02 = np.array([7.36, 7.36, 7.36, 3.68, 3.68])
SE02 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES02 = np.minimum(EV02, SE02)
# print(ES02)

EV03 = np.array([7.36, 7.36, 7.36, 7.36, 3.68])
SE03 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES03 = np.minimum(EV03, SE03)
# print(ES03)

EVSE = np.array([ES01, ES02, ES03])
# print(EVSE)

##### TGC: Central Controller

In [ ]:
# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# calclate the maximum charge possible for each session
session_max_charge = np.sum(pt * EVSE, axis=1)

# desired charge for each session
desired_charge = np.array([350, 200, 350])

# realistic charge for each session
realistic_charge = np.minimum(session_max_charge, desired_charge)


In [ ]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(
        w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
        for j in model.J
    ) + sum(
        (
            1
            - sum(pt[j - 1] * model.x[i, j] for j in model.J)
            / model.realistic_charge[i]
        )
        for i in model.I
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power EVSE 1: {[round(tgc.x[1, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 2: {[round(tgc.x[2, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 3: {[round(tgc.x[3, x].value, 2) for x in range(1, 6)]}")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))

### 🚧 Pyomo-3

Dynamic pricing 

<img src="./images/anwb.png" width="1000">

#### Model

&\sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right)

$$
\begin{align*}
\text{minimize} \; \left( 
    \alpha \left( \frac{1}{M_u} \sum_{j} w_{j} \left( 1 - \frac{\sum_{i} x_{ij}}{ex_{j}} \right) \right) + 
    \beta \left( \frac{1}{M_d} \sum_{i} \left(1 - \frac{fc_{i}}{rc_{i}}  \right) \right) + 
    \gamma \left( \frac{1}{M_c} \sum_{j} C[j] \times V[j] \right) 
\right)\\\\
\text{where} \; 
& w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad 
fc_{i} = \sum_{j=1}^{n} pt_{j} x_{ij}, \quad 
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}), \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$


$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \left( \frac{1}{M_u} \sum_{j} w_{j} \left( 1 - \frac{\sum_{i} x_{ij}}{ex_{j}} \right) \right) + 
    \beta \left( \frac{1}{M_d} \sum_{i} \left(1 - \frac{fc_{i}}{rc_{i}}  \right) \right) + 
    \gamma \left( \frac{1}{M_c} \sum_{j} C[j] \times V[j] \right) 
\right)\\\\
\text{where} \; & w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad 
fc_{i} = \sum_{j=1}^{n} pt_{j} x_{ij}, \quad 
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}), \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

Finally we will extend the ``pyomo-2`` model to also take the energy cost 💰 per period into account



##### Objective Function

Modifying the ``pyomo-2`` model results in the following ``Objective function`` to be minimized:  
<br>
<br>

$$
\begin{align*}
\text{minimize} \; & \left( 
    \alpha \sum_{j=1}^{n} w_{j} \left( 1 - \frac{\sum_{i=1}^{m} x_{ij}}{ex_{j}} \right) + 
    \frac{\beta}{m} \sum_{i=1}^{m} \left(1 - \frac{fc_{i}}{rc_{i}}  \right)  + 
    \frac{\gamma}{M_c} \sum_{j=1}^{n} C[j] \times V[j]  
\right)\\\\
\text{where} \; & w_{j} = \frac{pt_{j}}{\sum_{k=1}^{n} pt_{k}}, \quad 
fc_{i} = \sum_{j=1}^{n} pt_{j} x_{ij}, \quad 
rc_{i} = \sum_{j=1}^{n} pt_{j} \min(ev_{ij}, se_{ij}), \\\\
& \forall j,k \in \{1, 2, \ldots, n\}, \; \forall i \in \{1, 2, \ldots, m\}
\end{align*}
$$

$$
\begin{array}{rllll}
\text{Note: } & \\
& i \text{ represents a specific charge session EV/SE with } m \text{ being the maximum parallel charge sessions possible.} \\
& j \text{ represents a specific time period out of } n \text{ time periods in the forecast horizon.} \\\\
& {M_c} \text{ Maximum energy cost} \\\\
& \alpha \text{ imporance to optimize utilization} \\
& \beta \text{ imporance to optimize desired customer demand} \\
& \gamma \text{ imporance to optimize energy cost}
\end{array} 
$$

##### Decision Variables

The charging power for EV/SE $i$ at time slot $j$ are the decision variables, $x_{ij}$ 

##### Constraints

The charging power to be used ($x_{ij}$) in kW is subject to 5 constraints:

1. always needs to be positive, Vehicle to Grid (V2G) is **not** allowed.
1. less or equal than the Enexis maximum power output ($ex_j$) of Grid Connection. 
1. less or equal than the maximum power input ($ev_j$) of the EV onboard charger (obc)
1. less or equal than the maximum power output ($se_j$) of the EVSE.
1. the ``final charge`` $fc_{i}$ of EV $i$  should be less or equal to the ``realistic charge`` $rc_{i}$  
(which is less or equal to the ``desired charge`` $dc_{i}$)


$$
\begin{array}{rllll}
\text{s.t.} & x_{ij} \geq 0  &\\
\sum_{i=1}^{m} & x_{ij} \leq ex_{j} &\\
& x_{ij} \leq ev_{ij} &\\
& x_{ij} \leq se_{ij} &\\
\sum_{j=1}^{n} & pt_{j} x_{ij} \leq rc_{i} & 
\end{array} 
$$



$$
\begin{array}{ll}
\forall i \in \{1, 2, \ldots, m,\} & \text{i =  charge session,} & \text{m = 3}  \\
\forall j \in \{1, 2, \ldots, n\} &\text{j =  time period,} & \text{n = 5}
\end{array}
$$

This example relates to the ``pyomo_2`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)

#### Excel: results (sheet ``pyomo_3``) 

<img src="./images/pyomo_3.png" width="1000">

#### Code & Results

##### Sessions: EV/EVSE

In [4]:
import numpy as np

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

# --------------------------------------------------------------------------
# Data from each Session EV & SE

EV_MPI = 7.36  # EV max Power input for each time period
SE_MPO = 7.36  # EVSE max Power output for each time period

# For each Session EV & SE Compare the arrays and keep the minimum
EV01 = np.array([3.68, 7.36, 7.36, 3.68, 0.00])
SE01 = np.repeat(SE_MPO, n)
SE01 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES01 = np.minimum(EV01, SE01)
# print(ES01)

EV02 = np.array([7.36, 7.36, 7.36, 3.68, 3.68])
SE02 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES02 = np.minimum(EV02, SE02)
# print(ES02)

EV03 = np.array([7.36, 7.36, 7.36, 7.36, 3.68])
SE03 = np.array([5.00, 7.36, 7.36, 7.36, 7.36])
ES03 = np.minimum(EV03, SE03)
# print(ES03)

EVSE = np.array([ES01, ES02, ES03])
# print(EVSE)

[[3.68 7.36 7.36 3.68 0.  ]
 [5.   7.36 7.36 3.68 3.68]
 [5.   7.36 7.36 7.36 3.68]]


##### TGC: Central Controller

In [2]:
# Create the weights w[j] for the parking 
pt = np.array([40, 10, 20, 10, 10])
w = pt/np.sum(pt)

# calclate the maximum charge possible for each session
session_max_charge = np.sum(pt * EVSE, axis=1)

# desired charge for each session
desired_charge = np.array([350, 200, 350])

# realistic charge for each session
realistic_charge = np.minimum(session_max_charge, desired_charge)


In [3]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *
import numpy as np

m = 3  # number of EVSEs
n = 5  # number of time periods in the future

EX_MPO = 14.72  # enexis max Power output for each time period (constant)


# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(
        w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
        for j in model.J
    ) + sum(
        (
            1
            - sum(pt[j - 1] * model.x[i, j] for j in model.J)
            / model.realistic_charge[i]
        )
        for i in model.I
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# tgc.pprint()
print("\n")
print(f"Power EVSE 1: {[round(tgc.x[1, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 2: {[round(tgc.x[2, x].value, 2) for x in range(1, 6)]}")
print(f"Power EVSE 3: {[round(tgc.x[3, x].value, 2) for x in range(1, 6)]}")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))



Power EVSE 1: [3.23, 7.36, 7.36, 0.0, 0.0]
Power EVSE 2: [5.0, 0.0, 0.0, 0.0, 0.0]
Power EVSE 3: [3.23, 7.36, 7.36, 0.0, 0.0]

Objective Outcome =  0.3207
